### Classification using Keras 
Note: Keras uses TensorFlow by default. You can switch backend to Theano (see https://keras.io/backend/ for details)

In [190]:
import keras 
keras.__version__

'2.0.4'

In [191]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, log_loss, classification_report
from sklearn.metrics import confusion_matrix , mean_squared_error, r2_score
from sklearn.datasets import load_iris, load_boston

# Import `Sequential` from `keras.models`
from keras.models import Sequential
# Import `Dense` from `keras.layers`
from keras.layers import Dense, Activation, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import SGD, RMSprop, Adam

iris = load_iris()
X = iris.data
y = iris.target 
X = X.astype('float32')
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, train_size = 0.7, random_state=707)

nb_classes = len(set(y_train))
print(nb_classes, 'classes')

dims = X_train.shape[1]
print(dims, 'dims')#

y_test_c = y_test
# Convert labels to categorical one-hot encoding (there is also a sklear wrapper)
y_train = keras.utils.to_categorical(y_train, num_classes=3)
y_test = keras.utils.to_categorical(y_test, num_classes=3)

3 classes
4 dims


In [192]:
#### https://keras.io/getting-started/sequential-model-guide/
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=dims))
model.add(Dense(nb_classes, activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_112 (Dense)            (None, 100)               500       
_________________________________________________________________
dense_113 (Dense)            (None, 3)                 303       
Total params: 803
Trainable params: 803
Non-trainable params: 0
_________________________________________________________________


In [193]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1) 
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs=50, 
          batch_size=15, verbose=True, callbacks=[early_stop]) #callbacks=[best_model, early_stop]

Train on 105 samples, validate on 45 samples
Epoch 1/50
105/105 [==============================] - 0s - loss: 1.2239 - acc: 0.3333 - val_loss: 1.0249 - val_acc: 0.3333
Epoch 2/50
105/105 [==============================] - 0s - loss: 0.9736 - acc: 0.5143 - val_loss: 0.9047 - val_acc: 0.6667
Epoch 3/50
105/105 [==============================] - 0s - loss: 0.8763 - acc: 0.6476 - val_loss: 0.8368 - val_acc: 0.6889
Epoch 4/50
105/105 [==============================] - 0s - loss: 0.8109 - acc: 0.7143 - val_loss: 0.7790 - val_acc: 0.7111
Epoch 5/50
105/105 [==============================] - 0s - loss: 0.7711 - acc: 0.7048 - val_loss: 0.7407 - val_acc: 0.9556
Epoch 6/50
105/105 [==============================] - 0s - loss: 0.7299 - acc: 0.7238 - val_loss: 0.7052 - val_acc: 0.9556
Epoch 7/50
105/105 [==============================] - 0s - loss: 0.6952 - acc: 0.8762 - val_loss: 0.6697 - val_acc: 0.6667
Epoch 8/50
105/105 [==============================] - 0s - loss: 0.6644 - acc: 0.7619 - val_lo

In [194]:
y_pred = model.predict(X_test)
score = model.evaluate(X_test, y_test,verbose=1)
print("\n score: ",score)

32/45 [====================>.........] - ETA: 0s
 score:  [0.20887947115633224, 1.0]


In [195]:
y_pred_c = model.predict_classes(X_test)
# Confusion matrix
confusion_matrix(y_test_c, y_pred_c)

32/45 [====================>.........] - ETA: 0s

array([[15,  0,  0],
       [ 0, 15,  0],
       [ 0,  0, 15]])

In [196]:
# Precision 
#precision_score(y_test, y_pred)
log_loss(y_test, y_pred)

0.20887947768076426

In [197]:
y_pred_c[:10]

array([2, 1, 2, 0, 0, 1, 2, 0, 1, 2], dtype=int64)

In [198]:
y_pred[:10]

array([[  2.27587458e-04,   1.83421224e-01,   8.16351116e-01],
       [  9.05075390e-03,   7.75601208e-01,   2.15347961e-01],
       [  5.54333616e-04,   3.26268613e-01,   6.73177004e-01],
       [  9.91156936e-01,   8.73902813e-03,   1.04094273e-04],
       [  9.93084908e-01,   6.86344272e-03,   5.16137516e-05],
       [  4.08209860e-02,   7.71686494e-01,   1.87492505e-01],
       [  9.18427104e-05,   1.42435327e-01,   8.57472837e-01],
       [  9.84468520e-01,   1.53532578e-02,   1.78321672e-04],
       [  6.98237168e-03,   7.40357816e-01,   2.52659768e-01],
       [  6.65806991e-04,   3.56565475e-01,   6.42768741e-01]], dtype=float32)